# Mais sobre `SELECT `: agrupamento, subqueries

## Introdução

Vamos continuar nossa investigação sobre o comando `SELECT`. Desta vez usaremos a base de dados 'sakila', uma famosa base de dados de teste construída pelos autores do MySQL. 

Abra a URL [`https://dev.mysql.com/doc/sakila/en/`](https://dev.mysql.com/doc/sakila/en/) para conhecer melhor esta base de dados, que modela uma locadora de DVDs (riam, vocês também serão velhos um dia!). Se você não possui esta base, siga as instruções de instalação na página.

Vamos também construir nosso objeto auxiliar para conectar com a base de dados, como fizemos na última aula.

In [52]:
from functools import partial
import mysql.connector

connection = mysql.connector.connect(
    host='localhost',
    user='megadados',
    password='megadados',
    database='sakila',
    sql_mode='only_full_group_by',
)


def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)


db = partial(run_db_query, connection)

## Aquecimento

Quais os nomes das categorias de filme?

In [3]:
db('SELECT name FROM category')

Executando query:
('Action',)
('Animation',)
('Children',)
('Classics',)
('Comedy',)
('Documentary',)
('Drama',)
('Family',)
('Foreign',)
('Games',)
('Horror',)
('Music',)
('New',)
('Sci-Fi',)
('Sports',)
('Travel',)


Quais atores tem as iniciais "J.D."?

In [4]:
db('''
SELECT 
    first_name, last_name
FROM 
    actor
WHERE
    last_name like "D%" AND first_name like "J%"
''')

Executando query:
('JENNIFER', 'DAVIS')
('JUDY', 'DEAN')
('JODIE', 'DEGENERES')
('JULIANNE', 'DENCH')


Inventório lista as cópias físicas: DVD e loja onde está alocado. Porque posso ter várias cópias do mesmo filme e naquela loja (por isso não pode ser PK)

Endereço pode ter 2.

Liste as cidades brasileiras presentes na base de dados.

In [6]:
db('''
SELECT 
    city , country
FROM 
    country
    INNER JOIN city USING (country_id)
WHERE
    country  = 'Brazil'

''')

Executando query:
('Alvorada', 'Brazil')
('Angra dos Reis', 'Brazil')
('Anpolis', 'Brazil')
('Aparecida de Goinia', 'Brazil')
('Araatuba', 'Brazil')
('Bag', 'Brazil')
('Belm', 'Brazil')
('Blumenau', 'Brazil')
('Boa Vista', 'Brazil')
('Braslia', 'Brazil')
('Goinia', 'Brazil')
('Guaruj', 'Brazil')
('guas Lindas de Gois', 'Brazil')
('Ibirit', 'Brazil')
('Juazeiro do Norte', 'Brazil')
('Juiz de Fora', 'Brazil')
('Luzinia', 'Brazil')
('Maring', 'Brazil')
('Po', 'Brazil')
('Poos de Caldas', 'Brazil')
('Rio Claro', 'Brazil')
('Santa Brbara dOeste', 'Brazil')
('Santo Andr', 'Brazil')
('So Bernardo do Campo', 'Brazil')
('So Leopoldo', 'Brazil')
('Sorocaba', 'Brazil')
('Vila Velha', 'Brazil')
('Vitria de Santo Anto', 'Brazil')


*Quantas* cidades brasileiras tem na base de dados?

In [7]:
# Possibilidade de repetição, colocar distinct
db('''
SELECT 
    COUNT(city_id)
FROM 
    country
    INNER JOIN city USING (country_id)
WHERE
    country  = 'Brazil'

''')


Executando query:
(28,)


Liste os filmes do ator (fictício) "Dan Harris"

In [8]:
db('''
SELECT 
   film.title
FROM 
    film_actor
    INNER JOIN actor USING (actor_id)
    INNER JOIN film USING(film_id)    
WHERE
    actor.first_name = 'Dan' 
    AND actor.last_name = 'Harris'
''')

Executando query:
('BEDAZZLED MARRIED',)
('BOONDOCK BALLROOM',)
('DESTINY SATURDAY',)
('DIVINE RESURRECTION',)
('EYES DRIVING',)
('FELLOWSHIP AUTUMN',)
('GHOST GROUNDHOG',)
('GROOVE FICTION',)
('HILLS NEIGHBORS',)
('HOLIDAY GAMES',)
('INDEPENDENCE HOTEL',)
('INSIDER ARIZONA',)
('JADE BUNCH',)
('LIES TREATMENT',)
('MONTEREY LABYRINTH',)
('REUNION WITCHES',)
('RUN PACIFIC',)
('SCHOOL JACKET',)
('SEVEN SWARM',)
('SIEGE MADRE',)
('STEERS ARMAGEDDON',)
('STRAIGHT HOURS',)
('SUMMER SCARFACE',)
('SUPERFLY TRIP',)
('TITANIC BOONDOCK',)
('TITANS JERK',)
('VANISHING ROCKY',)
('WATERSHIP FRONTIER',)


In [9]:
#Resultado do monstrinho dou um inner join com outra tabela
db('''
SELECT 
    title
FROM 
    actor
    INNER JOIN film_actor USING (actor_id)
    INNER JOIN film USING(film_id)    
WHERE
    actor.first_name = 'Dan' 
    AND actor.last_name = 'Harris'
''')

Executando query:
('BEDAZZLED MARRIED',)
('BOONDOCK BALLROOM',)
('DESTINY SATURDAY',)
('DIVINE RESURRECTION',)
('EYES DRIVING',)
('FELLOWSHIP AUTUMN',)
('GHOST GROUNDHOG',)
('GROOVE FICTION',)
('HILLS NEIGHBORS',)
('HOLIDAY GAMES',)
('INDEPENDENCE HOTEL',)
('INSIDER ARIZONA',)
('JADE BUNCH',)
('LIES TREATMENT',)
('MONTEREY LABYRINTH',)
('REUNION WITCHES',)
('RUN PACIFIC',)
('SCHOOL JACKET',)
('SEVEN SWARM',)
('SIEGE MADRE',)
('STEERS ARMAGEDDON',)
('STRAIGHT HOURS',)
('SUMMER SCARFACE',)
('SUPERFLY TRIP',)
('TITANIC BOONDOCK',)
('TITANS JERK',)
('VANISHING ROCKY',)
('WATERSHIP FRONTIER',)


Quais filmes estão alugados por Florence Woods?

In [10]:
db('''
SELECT 
   film.title, customer.first_name, customer.last_name
FROM 
    inventory
    INNER JOIN film USING (film_id)
    INNER JOIN rental USING(inventory_id)
    INNER JOIN customer USING(customer_id)
WHERE 
    customer.first_name = 'Florence'
    AND customer.last_name = 'Woods'
    AND rental.return_date IS NULL
''')

Executando query:
('CLUB GRAFFITI', 'FLORENCE', 'WOODS')
('BLADE POLISH', 'FLORENCE', 'WOODS')


In [11]:
db('''
SELECT 
    title, first_name, last_name
FROM 
    customer
    INNER JOIN rental USING (customer_id)
    INNER JOIN inventory USING(inventory_id)
    INNER JOIN film USING(film_id)
WHERE 
    first_name = 'Florence'
    AND last_name = 'Woods'
    AND return_date IS NULL
''')

Executando query:
('CLUB GRAFFITI', 'FLORENCE', 'WOODS')
('BLADE POLISH', 'FLORENCE', 'WOODS')


Para quais línguas não tem nenhum filme na locadora? (Dica: use `LEFT OUTER JOIN`.)

In [12]:
db('''
SELECT 
    language.name
FROM
    language
    LEFT OUTER JOIN film ON language.language_id = film.language_id
WHERE 
    film.film_id IS NULL
''')

Executando query:
('Italian',)
('Japanese',)
('Mandarin',)
('French',)
('German',)


In [13]:
db('''
SELECT 
    name
FROM
    language
    LEFT OUTER JOIN film USING (language_id) 
WHERE 
    film.film_id IS NULL
''')

Executando query:
('Italian',)
('Japanese',)
('Mandarin',)
('French',)
('German',)


As vezes desejamos consultar quais os valores distintos de uma coluna. Para isso usamos o qualificador `DISTINCT`. 

Por exemplo: Quais os anos de lançamento dos filmes da base? 

In [14]:
# Vai aparecer muitas cópias de '(2006,)'
db('SELECT release_year FROM film LIMIT 20')

Executando query:
(2006,)
(2006,)
(2006,)
(2006,)
(2006,)
(2006,)
(2006,)
(2006,)
(2006,)
(2006,)
(2006,)
(2006,)
(2006,)
(2006,)
(2006,)
(2006,)
(2006,)
(2006,)
(2006,)
(2006,)


Ops, parece que não tem muita variedade nesta base! Usando `DISTINCT` podemos limpar esse resultado:

In [15]:
db('SELECT DISTINCT release_year FROM film')

Executando query:
(2006,)


### Praticando

Quais clientes estão alugando um DVD agora?

In [16]:
db('''
SELECT 
    DISTINCT customer.first_name, customer.last_name
FROM 
    customer
    INNER JOIN rental USING (customer_id)
WHERE 
    return_date IS NULL
ORDER BY
    first_name, last_name
''')

Executando query:
('ADRIAN', 'CLARY')
('ALBERT', 'CROUSE')
('ALBERTO', 'HENNING')
('ALICIA', 'MILLS')
('ALLAN', 'CORNISH')
('ALLEN', 'BUTTERFIELD')
('ALLISON', 'STANLEY')
('ANA', 'BRADLEY')
('ANDY', 'VANHORN')
('ANGELA', 'HERNANDEZ')
('ANNA', 'HILL')
('ANNETTE', 'OLSON')
('APRIL', 'BURNS')
('BARRY', 'LOVELACE')
('BECKY', 'MILES')
('BERNARD', 'COLBY')
('BETH', 'FRANKLIN')
('BETTY', 'WHITE')
('BEVERLY', 'BROOKS')
('BILL', 'GAVIN')
('BRANDON', 'HUEY')
('BRENT', 'HARKINS')
('CARMEN', 'OWENS')
('CAROLYN', 'PEREZ')
('CASSANDRA', 'WALTERS')
('CATHY', 'SPENCER')
('CECIL', 'VINES')
('CHARLIE', 'BESS')
('CHRIS', 'BROTHERS')
('CHRISTIAN', 'JUNG')
('CHRISTINE', 'ROBERTS')
('CLAUDIA', 'FULLER')
('CLINTON', 'BUFORD')
('COLLEEN', 'BURTON')
('CORY', 'MEEHAN')
('COURTNEY', 'DAY')
('CRAIG', 'MORRELL')
('CURTIS', 'IRBY')
('CYNTHIA', 'YOUNG')
('DAISY', 'BATES')
('DARRYL', 'ASHCRAFT')
('DARYL', 'LARUE')
('DERRICK', 'BOURQUE')
('DIANNE', 'SHELTON')
('DUSTIN', 'GILLETTE')
('DWAYNE', 'OLVERA')
('EDWARD', 'BAU

## Agrupamento

Uma das características mais valiosas de banco de dados é o *agrupamento*. Podemos agrupar os resultados de uma query indicando uma coluna cujos valores serão usados para agrupar os dados.

Por exemplo, considere a seguinte tabela, que chamaremos de 'vendas':

| id | id_item | item | preco |
|--|--|--|--|
| 1 | 1 | A | 5 |
| 2 | 2 | B | 6 |
| 3 | 1 | A | 3 |
| 4 | 3 | C | 7 |
| 5 | 3 | C | 5 |
| 6 | 1 | A | 2 |

Se agruparmos pela coluna 'id_item' teremos 3 conjuntos de resultados:

id_item = 1:

| id | id_item | item | preco |
|--|--|--|--|
| 1 | 1 | A | 5 |
| 3 | 1 | A | 3 |
| 6 | 1 | A | 2 |

id_item = 2:

| id | id_item | item | preco |
|--|--|--|--|
| 2 | 2 | B | 6 |

id_item = 3:

| id | id_item | item | preco |
|--|--|--|--|
| 4 | 3 | C | 7 |
| 5 | 3 | C | 5 |

É como se tivessemos uma lista de tabelas! Isso não é permitido em SQL. Temos que 'resumir' a informação de cada uma das tabelas a uma linha só, o que significa que, para cada coluna, devemos escolher uma dessas opções:
- Resumir a informação da coluna usando uma função de grupo. Podemos somar, tirar a média, contar itens, concatená-los em uma única string, entre outras;
- Para colunas que se relacionam 1 para 1 com a coluna de agrupamento (como a coluna de agrupamento em si, ou a coluna 'item' neste exemplo), manter este valor. Isso acontece frequentemente quando fazemos `JOIN`.
- Não incluir a coluna, caso contrário.

Neste exemplo, podemos tomar a seguinte decisão para cada coluna:
- id: descartar
- id_item: manter valor
- item: manter valor
- preco: vamos calcular a soma dos valores, e renomear esta informação para 'total'

Com isso, obtemos a seguinte tabela:

| id_item | item | total |
|--|--|--|
| 1 | A | 10 |
| 2 | B | 6 |
| 3 | C | 12 |

Por fim, se não queremos id_item, ficamos com a seguinte tabela:

| item | total |
|--|--|
| A | 10 |
| B | 6 |
| C | 12 |

Para obter essa tabela podemos usar o seguinte comando SQL:

```SQL
SELECT 
    item, SUM(preco) as total 
FROM 
    vendas
GROUP BY
    id_item
```

Consulte o capítulo 9 do seu livro texto para conhecer mais sobre agrupamentos.

### Praticando

Quais os 10 atores que mais apareceram em filmes?

In [32]:
db('''
SELECT 
    first_name, last_name, COUNT(first_name) as aparicao
FROM
    film
    INNER JOIN film_actor USING (film_id)
    INNER JOIN actor USING (actor_id)
GROUP BY
    actor_id
ORDER BY
    aparicao DESC
LIMIT
    10
''')


Executando query:
('GINA', 'DEGENERES', 42)
('WALTER', 'TORN', 41)
('MARY', 'KEITEL', 40)
('MATTHEW', 'CARREY', 39)
('SANDRA', 'KILMER', 37)
('SCARLETT', 'DAMON', 36)
('VAL', 'BOLGER', 35)
('VIVIEN', 'BASINGER', 35)
('GROUCHO', 'DUNST', 35)
('UMA', 'WOOD', 35)


In [33]:
db('''
SELECT 
    first_name, last_name, COUNT(film_id) as aparicao
FROM
    actor
    INNER JOIN film_actor USING (actor_id)
GROUP BY
    actor_id
ORDER BY
    aparicao DESC
LIMIT
    10
''')

Executando query:
('GINA', 'DEGENERES', 42)
('WALTER', 'TORN', 41)
('MARY', 'KEITEL', 40)
('MATTHEW', 'CARREY', 39)
('SANDRA', 'KILMER', 37)
('SCARLETT', 'DAMON', 36)
('VAL', 'BOLGER', 35)
('VIVIEN', 'BASINGER', 35)
('GROUCHO', 'DUNST', 35)
('UMA', 'WOOD', 35)


In [34]:
db('''
SELECT 
    actor.actor_id,
    actor.first_name,
    actor.last_name,
    COUNT(actor_id) AS film_count
FROM 
    actor
    INNER JOIN film_actor USING(actor_id)
GROUP BY 
    actor_id
ORDER BY
    film_count DESC
LIMIT 
    10
''')




Executando query:
(107, 'GINA', 'DEGENERES', 42)
(102, 'WALTER', 'TORN', 41)
(198, 'MARY', 'KEITEL', 40)
(181, 'MATTHEW', 'CARREY', 39)
(23, 'SANDRA', 'KILMER', 37)
(81, 'SCARLETT', 'DAMON', 36)
(37, 'VAL', 'BOLGER', 35)
(158, 'VIVIEN', 'BASINGER', 35)
(106, 'GROUCHO', 'DUNST', 35)
(13, 'UMA', 'WOOD', 35)


## Pipeline do comando `SELECT`

Uma versão mais completa do `SELECT` (mas não inteiramente completa - consulte o manual do MySQL) é vista abaixo:

```
SELECT [DISTINCT] <select_header> 
FROM <source_tables>
WHERE <filter_expression>
GROUP BY <grouping_expressions>
HAVING <filter_expression>
ORDER BY <ordering_expressions>
LIMIT <count> 
OFFSET <count>
```

Você já deve ter percebido que o comando `SELECT` tem uma sequência própria de avaliação. Por exemplo, para saber quais filmes custam mais que 3 dinheiros, podemos escrever:

In [35]:
db('''
SELECT 
    COUNT(f.rental_rate)
FROM
    film f
WHERE
    f.rental_rate > 3
''')

Executando query:
(336,)


Observe que o 'apelido' f para a tabela 'film' é definido na cláusula `FROM`, mas usado em `SELECT` e também em `WHERE`.

A ordem de execução do comando `SELECT` é aproximadamente como segue:

1. `FROM <source_tables>`: indica as tabelas que serão usadas nesta query e, conceitualmente, combina estas tabelas através de *produto cartesiano* em uma grande tabela. (Note o termo "*conceitualmente*" que usei: em termos de implementação da query este produto cartesiano raramente é construído.)

2. `WHERE <filter_expression>`: filtra linhas.

3. `GROUP BY <grouping_expressions>`: agrupa conjuntos de linhas.

4. `SELECT <select_heading>`: escolha de colunas e de agregados.

5. `HAVING <filter_expression>`: outra filtragem, esta aplicada apenas **depois** da agregação. Pode usar resultados do processo de agregação. Obriga o uso de `GROUP BY`.

6. `DISTINCT`: Elimina linhas duplicadas.

7. `ORDER BY`: ordena as linhas do resultado.

8. `OFFSET <count>`: Pula linhas do resultado. Requer LIMIT.

9. `LIMIT <count>`: Mantém apenas um número máximo de linhas.

Esta sequencia também serve como dica de como projetar uma query! 
- Comece identificando as tabelas que você deseja usar
- Monte o filtro de linhas, incluindo critérios de `JOIN`
- Agrupe
- Selecione colunas e aplique funções de agregação, conforme necessário
- Filtre com `HAVING`, agora que temos agregação
- O resto é mais fácil, aplique conforme requerido

## `WHERE` versus `HAVING`

Conforme visto acima, temos a cláusula `HAVING` para fazer filtragens *APÓS* agregação. Para que serve isso? Por exemplo, suponha que queremos saber quais atores não compartilham seu sobrenome com nenhum outro ator. Podemos usar a *query* a seguir:

In [40]:
# MIN usado para pegar o menor deles, depois de dar group by pega só aquele primeiro nome
# A gente quer saber quais são os atotres que não compartilham seu sobrenome com nenhum ator, então quero descobrir 
# quais atores, onde o número de atores com aquele sobrenome é 1.

#Faço o group by com o last name e toda vez que agrupo, agrego. Mantenho linahs resumo onde o count do first_name é igual a 1.
#


db('''
SELECT 
    MIN(a.first_name) as fst_name, a.last_name
FROM 
    actor a 
GROUP BY 
    a.last_name
HAVING
    COUNT(a.first_name) = 1
ORDER BY
    a.last_name, fst_name
''')

Executando query:
('ANGELINA', 'ASTAIRE')
('RUSSELL', 'BACALL')
('HARRISON', 'BALE')
('RENEE', 'BALL')
('JULIA', 'BARRYMORE')
('VIVIEN', 'BASINGER')
('VIVIEN', 'BERGEN')
('LIZA', 'BERGMAN')
('CUBA', 'BIRCH')
('KEVIN', 'BLOOM')
('CHRIS', 'BRIDGES')
('LAURENCE', 'BULLOCK')
('MATTHEW', 'CARREY')
('GREG', 'CHAPLIN')
('RUSSELL', 'CLOSE')
('FRED', 'COSTNER')
('SIDNEY', 'CROWE')
('JUDE', 'CRUISE')
('RALPH', 'CRUZ')
('SCARLETT', 'DAMON')
('FRANCES', 'DAY-LEWIS')
('SYLVESTER', 'DERN')
('ALAN', 'DREYFUSS')
('GROUCHO', 'DUNST')
('CHRISTIAN', 'GABLE')
('MERYL', 'GIBSON')
('PARKER', 'GOLDBERG')
('ADAM', 'GRANT')
('MEG', 'HAWKE')
('GEOFFREY', 'HESTON')
('HARVEY', 'HOPE')
('ANGELA', 'HUDSON')
('CARMEN', 'HUNT')
('WHOOPI', 'HURT')
('WOODY', 'JOLIE')
('KIRK', 'JOVOVICH')
('MATTHEW', 'LEIGH')
('JOHNNY', 'LOLLOBRIGIDA')
('GRETA', 'MALDEN')
('ED', 'MANSFIELD')
('ELVIS', 'MARX')
('MORGAN', 'MCDORMAND')
('TOM', 'MIRANDA')
('BETTE', 'NICHOLSON')
('KENNETH', 'PESCI')
('OLYMPIA', 'PFEIFFER')
('GARY', 'PHOENIX'

## Praticando

Liste a duração média dos filmes na categoria 'Drama'. Aqui o `GROUP BY` é necessário?

In [46]:
db('''
SELECT 
    AVG(length)
FROM 
    film
    INNER JOIN film_category USING (film_id)
    INNER JOIN category USING (category_id)
    category.name = 'Drama'
''')


Executando query:
(Decimal('120.8387'),)


Liste o nome da categoria e a duração média dos filmes por categoria.

In [47]:
# Agrupar categoria e agregar com AVG

db('''
SELECT 
    category.name, AVG(length)
FROM 
    film
    INNER JOIN film_category USING (film_id)
    INNER JOIN category USING (category_id)
GROUP BY
    category.name
ORDER BY
    AVG(length) DESC
''')

Executando query:
('Sports', Decimal('128.2027'))
('Games', Decimal('127.8361'))
('Foreign', Decimal('121.6986'))
('Drama', Decimal('120.8387'))
('Comedy', Decimal('115.8276'))
('Family', Decimal('114.7826'))
('Music', Decimal('113.6471'))
('Travel', Decimal('113.3158'))
('Horror', Decimal('112.4821'))
('Classics', Decimal('111.6667'))
('Action', Decimal('111.6094'))
('New', Decimal('111.1270'))
('Animation', Decimal('111.0152'))
('Children', Decimal('109.8000'))
('Documentary', Decimal('108.7500'))
('Sci-Fi', Decimal('108.1967'))


In [49]:
#name puro pois to agrupando pela chave primaria de categoria
db('''
SELECT 
    name, AVG(length)
FROM 
    category
    INNER JOIN film_category USING (category_id)
    INNER JOIN film USING (film_id)
GROUP BY
    category_id
''')

Executando query:
('Action', Decimal('111.6094'))
('Animation', Decimal('111.0152'))
('Children', Decimal('109.8000'))
('Classics', Decimal('111.6667'))
('Comedy', Decimal('115.8276'))
('Documentary', Decimal('108.7500'))
('Drama', Decimal('120.8387'))
('Family', Decimal('114.7826'))
('Foreign', Decimal('121.6986'))
('Games', Decimal('127.8361'))
('Horror', Decimal('112.4821'))
('Music', Decimal('113.6471'))
('New', Decimal('111.1270'))
('Sci-Fi', Decimal('108.1967'))
('Sports', Decimal('128.2027'))
('Travel', Decimal('113.3158'))


Liste o nome da categoria e a duração média dos filmes por categoria, apenas para categorias cuja duração média de filme excede 120 minutos.

In [50]:
#Having uso para depois de ter agrupado por categoria  e agregar com avg, quero apenas selecionar a duração : primo do where

db('''
SELECT 
    category.name, AVG(length)
FROM 
    film
    INNER JOIN film_category USING (film_id)
    INNER JOIN category USING (category_id)
GROUP BY
    category.name
HAVING
    AVG(length) > 120
ORDER BY
    AVG(length) DESC
''')

Executando query:
('Sports', Decimal('128.2027'))
('Games', Decimal('127.8361'))
('Foreign', Decimal('121.6986'))
('Drama', Decimal('120.8387'))


# Conclusão

Esta aula de hoje foi bastante densa! Dicas de estudo:

- Pratique no seu livro-texto, capítulo 9. Lembre-se que a base 'música' pode ser usada para praticar os comandos SQL vistos no livro.
- Tente criar queries que sirvam de exemplo para os conceitos do livro! A tarefa de criar exemplos é muito instrutiva!

**Leitura prévia**:
Para a próxima aula vamos continuar praticando, com os assuntos dos capítulos 10 e 11, prepare-se para a aula, ok?

Até a próxima!

Exercício) Qual a Query mais maligna que conseguimos imaginar. Inventar um ex com o maior número de joins, having, etc

Qual o país com o maior número de filmes alugados por pessoas que começam com o nome "L" cujo pertencem a categoria de menor duração média dentro desta seleção.


In [78]:
db('''
SELECT
    MIN(country), MIN(category.name), MIN(customer.first_name), AVG(length) as media
FROM 
    country
    INNER JOIN city USING (country_id)
    INNER JOIN address USING (city_id)
    INNER JOIN customer USING (address_id)
    INNER JOIN rental USING (customer_id)
    INNER JOIN inventory USING (inventory_id)
    INNER JOIN film USING (film_id)
    INNER JOIN film_category USING (film_id)
    INNER JOIN category USING(category_id)
WHERE 
    rental.return_date IS NULL 
    AND customer.first_name like "L%"
GROUP BY
    category.name
HAVING
    media = 51
ORDER BY
    AVG(length) ASC


''')

Executando query:
('South Africa', 'Sci-Fi', 'LOUISE', Decimal('51.0000'))


Qual o país em que as pessoas mais gastam com filmes da categoria 'Drama' que ainda não foram devolvidos e na qual a linguagem original do filme é francês?

Qual o país com o maior número de filmes alugados cujo pertencem a categoria de maior duração média

In [82]:
db('''
SELECT
    category_id, AVG(length) as avg_length
FROM 
    film_category
    INNER JOIN film USING (film_id)

GROUP BY
    category_id
ORDER BY
    avg_length DESC
LIMIT 
    1
''')


Executando query:
(15, Decimal('128.2027'))


In [ ]:
db(''' 




''')

In [51]:
connection.close()